In [1]:
import pandas as pd
import seaborn
import numpy as np

### Importing Data

In [2]:
df = pd.read_csv("data/ytdf3.csv")
df.head(5)

,Authorname,Comments
0,Radin Černý,Summary for those who are not fans of long vid...
1,Filipe Freire,More a linux starter tutorial than a docker on...
2,Troll Gamer,"Hello mosh, i am a 15 year old kid and was lea..."
3,Ash Moore,"Hi Mosh, I just want to say thank you so much ..."
4,Irfan Shaikh,Excellent beginners tutorial. You have not jum...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Authorname  100 non-null    object
 1   Comments    100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


### Data preprocessing 

Remove emojis and special characters

In [4]:
import re
def cleaning_comments(comment):
  comment = re.sub("[🤣|🤭|🤣|😁|🤭|❤️|👍|🏴|😣|😠|💪|🙏|😞|🌺|🌸|🌞|🌻|💐|💓|😥|💔|😪|😑|🏽|😢|😑|😇|💜|🪴|🙌🏻|🇨🇦|🕊|🕯|😭|😔|💙|🏼|✝|🇿]+",'',comment)
  comment = re.sub("[0-9]+","",comment)
  comment = re.sub("[\:|\@|\)|\*|\.|\$|\!|\?|\,|\%|\"|\(|\-|\”|\#|\!|\/|\«|\»|\&|\n|\'|\;|\!|<|>|\'|\’|\\\\]+"," ",comment)
  return comment

df['Comments'] = df['Comments'].apply(cleaning_comments)

Convert to lower case

In [5]:
lower = lambda x: x.lower()
df['Comments'] = df['Comments'].apply(lower)

Get the polarity with the stopwords

In [6]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

#Create scores col
df['Scores'] = df['Comments'].apply(lambda x: sid.polarity_scores(x))

#Create compund score
df['Compound'] = df['Scores'].apply(lambda score_dict: score_dict['compound'])

#label based on score
df['Polarity'] = df['Compound'].apply(lambda c: 'neutral' if -0.05 < c < 0.05 else ('negative' if c < -0.05 else 'positive'))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\wled3\AppData\Roaming\nltk_data...


Get the polarity without the stopwords

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop = stopwords.words('english')

#Create comments without stopwords col
df['comments wo stopwords'] = df['Comments'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

Add scores and polarity col without stopwords

In [8]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

df['Scores wo stopwords'] = df['comments wo stopwords'].apply(lambda x: sid.polarity_scores(x))

df['compound wo stopwords'] = df['Scores wo stopwords'].apply(lambda score_dict: score_dict['compound'])

df['polarity wo stopwords'] = df['compound wo stopwords'].apply(lambda c: 'neutral' if -0.05 < c < 0.05 else ('negative' if c < -0.05 else 'positive'))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\wled3\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
df["Length"] = df["Comments"].str.len()
df.head(5)

,Authorname,Comments,Scores,Compound,Polarity,comments wo stopwords,Scores wo stopwords,compound wo stopwords,polarity wo stopwords,Length
0,Radin Černý,summary for those who are not fans of long vid...,"{'neg': 0.0, 'neu': 0.862, 'pos': 0.138, 'comp...",0.9612,positive,summary fans long videos name video says docke...,"{'neg': 0.0, 'neu': 0.797, 'pos': 0.203, 'comp...",0.9186,positive,670
1,Filipe Freire,more a linux starter tutorial than a docker on...,"{'neg': 0.0, 'neu': 0.742, 'pos': 0.258, 'comp...",0.7869,positive,linux starter tutorial docker one good enough ...,"{'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'comp...",0.6124,positive,129
2,Troll Gamer,hello mosh i am a year old kid and was learn...,"{'neg': 0.0, 'neu': 0.75, 'pos': 0.25, 'compou...",0.9652,positive,hello mosh year old kid learning python videos...,"{'neg': 0.0, 'neu': 0.586, 'pos': 0.414, 'comp...",0.9652,positive,435
3,Ash Moore,hi mosh i just want to say thank you so much ...,"{'neg': 0.037, 'neu': 0.755, 'pos': 0.209, 'co...",0.9557,positive,hi mosh want say thank much providing valuable...,"{'neg': 0.06, 'neu': 0.543, 'pos': 0.397, 'com...",0.9774,positive,670
4,Irfan Shaikh,excellent beginners tutorial you have not jum...,"{'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'comp...",0.3291,positive,excellent beginners tutorial jumped complex th...,"{'neg': 0.0, 'neu': 0.764, 'pos': 0.236, 'comp...",0.5719,positive,140


Compare the polarity ratio with stopwords and without them

In [10]:
df['Polarity'].value_counts()

positive    92
neutral      4
negative     4
Name: Polarity, dtype: int64

In [11]:
df['polarity wo stopwords'].value_counts()

positive    93
neutral      4
negative     3
Name: polarity wo stopwords, dtype: int64